<h1>Voorbeeld data import met Pandas</h1>
<h4>Door B. Braam</h4>

We gaan hiermee de databases in archive gebruiken en we gaan daarvan de gegevens transformeren.
Ik begin met de juiste imports.

In [1]:
import pandas as pd

Nu gaan we met behulp van pandas een dataframe aanmaken en daar de csv appearances.csv inladen.<br/>
Vervolgens controleren we de bovenste 5 rijen in de csv (de table head.)

In [2]:
df = pd.read_csv('archive/appearances.csv')
print(df.head())

    appearance_id  game_id  player_id  player_club_id  player_current_club_id  \
0  2460589_237947  2460589     237947           11411                   16247   
1  2460503_150846  2460503     150846            4807                    2503   
2  2460507_136142  2460507     136142             271                     862   
3  2460508_153735  2460508     153735           18506                    4128   
4   2460509_27017  2460509      27017           10482                    3719   

         date       player_name competition_id  yellow_cards  red_cards  \
0  2014-07-01  Rumyan Hovsepyan            CLQ             0          0   
1  2014-07-03           Yoro Ly            ELQ             0          0   
2  2014-07-03   Armando Vajushi            ELQ             0          0   
3  2014-07-03  Marko Simonovski            ELQ             0          0   
4  2014-07-03   Dmitriy Khomich            ELQ             0          0   

   goals  assists  minutes_played  
0      0        0         

Vervolgens gaan we de dataframe verkleinen zodat we kunnen kijken hoeveel rode kaarten elke speler heeft. Wat we hiervoor nodig hebben is player_id en red_cards.